In [1]:
# Importing the functions
from lib.a_load import loader
from lib.b_clean_nan import nan_dropper
from lib.c_fill_means import nan_filler
from lib.d_transformer import *
from lib.ef_model import model

import sklearn
from sklearn.ensemble import RandomForestClassifier

## 1. Load and split the data

In [2]:
df = loader("sample_diabetes_mellitus_data.csv").load()
df

,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,...,ventilated_apache,wbc_apache,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus
0,214826,118,68.0,22.732803,0,Caucasian,M,180.30,Floor,Floor,...,0,14.1,0,0,0,0,0,0,0,1
1,246060,81,77.0,27.421875,0,Caucasian,F,160.00,Floor,Floor,...,1,12.7,0,0,0,0,0,0,0,1
2,276985,118,25.0,31.952749,0,Caucasian,F,172.70,Emergency Department,Accident & Emergency,...,0,NaN,0,0,0,0,0,0,0,0
3,262220,118,81.0,22.635548,1,Caucasian,F,165.10,Operating Room,Operating Room / Recovery,...,1,8.0,0,0,0,0,0,0,0,0
4,201746,33,19.0,NaN,0,Caucasian,M,188.00,NaN,Accident & Emergency,...,0,NaN,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,167042,140,NaN,51.952814,0,Caucasian,F,137.20,Floor,Floor,...,0,7.0,0,0,0,0,0,0,0,0
9996,165009,89,34.0,26.110113,0,Caucasian,F,175.26,Emergency Department,Accident & Emergency,...,0,9.5,0,0,0,0,0,0,0,0
9997,234052,137,30.0,23.774241,0,Native American,F,157.48,Emergency Department,Accident & Emergency,...,0,NaN,0,0,0,0,0,0,0,0
9998,249290,89,67.0,31.330708,0,Caucasian,F,157.48,Direct Admit,Accident & Emergency,...,0,6.4,0,0,0,0,0,0,0,0


In [3]:
train, test = loader("sample_diabetes_mellitus_data.csv").split()
print(f'{train.shape, test.shape}')

((7000, 52), (3000, 52))


## 2. Dropping missing values

In [4]:
train = nan_dropper(train).drop_nan(['age', 'gender', 'ethnicity'])
test = nan_dropper(test).drop_nan(['age', 'gender', 'ethnicity'])

## 3. Filling missing values

In [5]:
train = nan_filler(train).fill_means(['height', 'weight'])
test = nan_filler(test).fill_means(['height', 'weight'])

In [6]:
train.head()

,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,...,ventilated_apache,wbc_apache,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus
3194,220344,118,87.0,18.820156,0,Caucasian,M,165.1,Emergency Department,Accident & Emergency,...,1,10.0,0,0,0,0,0,0,0,0
1507,226441,118,32.0,20.859034,0,Caucasian,M,175.3,Operating Room,Other Hospital,...,1,33.9,0,0,0,0,0,0,0,0
3438,162338,81,44.0,53.515625,1,Hispanic,F,160.0,Operating Room,Operating Room / Recovery,...,1,13.7,0,0,0,0,0,0,0,1
9707,249592,140,74.0,26.615539,0,Caucasian,F,170.2,Emergency Department,Accident & Emergency,...,0,NaN,0,0,0,0,0,0,0,0
9547,184294,140,86.0,26.065657,0,Caucasian,M,175.3,Emergency Department,Accident & Emergency,...,0,NaN,0,0,0,0,0,0,0,0


## 4. Transforming the data

In [7]:
# Consider the following snippet is replacing the data in the dataframe passed to the function
# If you declare these to a variable, it will end up being a none type
# E.g. train = log(train, ['age', 'bmi']).apply_transform() will result in a none type

train = log(train, ['wbc_apache', 'ventilated_apache']).apply_transform()
train = levels(train, ['apache_2_diagnosis']).apply_transform()


test = log(test, ['wbc_apache', 'ventilated_apache']).apply_transform()
test = levels(test, ['apache_2_diagnosis']).apply_transform()


/Users/lucascosta/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/lucascosta/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [8]:
train.head()

,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,...,ventilated_apache,wbc_apache,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus
3194,220344,118,87.0,18.820156,0,Caucasian,M,165.1,Emergency Department,Accident & Emergency,...,0.0,1.000000,0,0,0,0,0,0,0,0
1507,226441,118,32.0,20.859034,0,Caucasian,M,175.3,Operating Room,Other Hospital,...,0.0,1.530200,0,0,0,0,0,0,0,0
3438,162338,81,44.0,53.515625,1,Hispanic,F,160.0,Operating Room,Operating Room / Recovery,...,0.0,1.136721,0,0,0,0,0,0,0,1
9707,249592,140,74.0,26.615539,0,Caucasian,F,170.2,Emergency Department,Accident & Emergency,...,-inf,NaN,0,0,0,0,0,0,0,0
9547,184294,140,86.0,26.065657,0,Caucasian,M,175.3,Emergency Department,Accident & Emergency,...,-inf,NaN,0,0,0,0,0,0,0,0


In [9]:
train.describe()

/Users/lucascosta/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


,encounter_id,hospital_id,age,bmi,elective_surgery,height,icu_id,pre_icu_los_days,readmission_status,weight,...,ventilated_apache,wbc_apache,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus
count,6553.000000,6553.000000,6553.000000,5404.000000,6553.000000,6553.000000,6553.000000,6553.000000,6553.0,6553.000000,...,6553.0,5154.000000,6553.000000,6553.000000,6553.000000,6553.000000,6553.000000,6553.000000,6553.000000,6553.000000
mean,212342.540211,103.529223,62.435678,30.018643,0.229666,170.159028,105.280787,0.634027,0.0,87.102780,...,-inf,1.000022,0.000305,0.019228,0.013429,0.046086,0.006867,0.005036,0.026553,0.236228
std,37906.369726,30.587139,16.628120,8.442535,0.420650,10.659472,16.972934,2.163882,0.0,23.485546,...,NaN,0.247433,0.017469,0.137335,0.115111,0.209687,0.082589,0.070790,0.160784,0.424796
min,147009.000000,4.000000,16.000000,14.844926,0.000000,137.200000,82.000000,-0.224306,0.0,38.600000,...,-inf,-0.045757,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,179371.000000,83.000000,53.000000,24.238815,0.000000,162.600000,92.000000,0.002083,0.0,72.300000,...,NaN,0.857332,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,212202.000000,118.000000,64.000000,28.424843,0.000000,170.200000,99.000000,0.010417,0.0,87.102780,...,NaN,0.995635,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,244881.000000,118.000000,75.000000,34.074977,0.000000,177.800000,114.000000,0.147222,0.0,97.200000,...,0.0,1.167317,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,279000.000000,198.000000,89.000000,67.814990,1.000000,195.590000,171.000000,49.523611,0.0,186.000000,...,0.0,1.660865,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## 5. Model

In [10]:
rf = RandomForestClassifier()

features = ['hepatic_failure', 'height', 'weight', 'elective_surgery', 'immunosuppression', 'leukemia','age','bmi']
target = ['diabetes_mellitus']

train = nan_dropper(train).drop_nan(features)
test = nan_dropper(test).drop_nan(features)

randomForest = model(train, rf, features, target)

/Users/lucascosta/Library/CloudStorage/OneDrive-Pessoal/Mestrado_DSDM/Term_1/Computing_for_Data_Science/hw6/hw6_computingds/lib/ef_model.py:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.model_fit = self.model.fit(X_train, y_train)


In [11]:
test['prediction'] = randomForest.predict(test)[:,1]

In [12]:
# Checking last two columns of test dataframe
test.iloc[:,-2:]

,diabetes_mellitus,prediction
7878,0,0.15
3224,0,0.69
1919,0,0.05
4835,0,0.36
4895,0,0.07
...,...,...
9613,0,0.72
5268,0,0.04
6772,0,0.19
4048,0,0.20


## 6. Model evaluation

In [13]:
# ROC AUC score

# Computing the results testing roc_auc

# packages required
from sklearn.metrics import roc_auc_score

# i. Compute the train and test roc_auc metric using roc_auc_score from sklearn
def roc_function(input, output):
    return roc_auc_score(input, output)

roc_metric_test = roc_function(test['diabetes_mellitus'], test['prediction'])

print("ROC AUC Score test ", roc_metric_test)

ROC AUC Score test  0.6279136086049398
